In [1]:
import numpy as np
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from shapely.geometry import Point

import swisslandstats as sls

In [2]:
vaud_ldf = sls.read_csv('data/vaud_lulc.csv')

In [3]:
with rasterio.open('data/swissALTI_reduced_res_lv95_3.tif') as src:
    elevation_affine = src.transform
    elevation_arr = src.read(1)
    elevation_crs = src.crs
    elevation_nodata = src.nodata

In [4]:
# Rectangular buffer in the DEM crs

vaud_gser = vaud_ldf.get_geoseries()

# for an square buffer, distance is half of the dataframe's resolution
buff_dist = vaud_ldf.res[0] / 2
buffer_gser = vaud_gser.to_crs(elevation_crs).apply(lambda x: x.buffer(buff_dist, cap_style=3))

I am densified (external_values, 321077 elements)


I am densified (external_values, 321077 elements)


In [12]:
elevation_stats = zonal_stats(
    buffer_gser, elevation_arr, affine=elevation_affine, nodata=elevation_nodata)

In [19]:
elevation_ser = pd.Series(map(lambda d: d['mean'], elevation_stats), name='elevation', index=vaud_ldf.index)

## Fill wrong values with mean neighbor elevation

In [20]:
# Convert negative (wrong values) to nan

elevation_ser[elevation_ser < 0] = np.nan

In [22]:
nan_point_gser = vaud_gser[elevation_ser.isna()]

In [23]:
def get_mean_neighbor_elevation(point, elevation_ser, spatial_index):
    return elevation_ser[list(spatial_index.nearest([point.x, point.y], 4, objects='raw'))].dropna().mean()

In [25]:
fillna_elevation_ser = nan_point_gser.apply(get_mean_neighbor_elevation, args=(elevation_ser, vaud_gser.sindex))
elevation_ser = elevation_ser.fillna(fillna_elevation_ser)

I am densified (external_values, 0 elements)


In [ ]:
elevation_ser.to_csv('data/vaud_elevation.csv', header=True)